In [10]:
import sys
import pandas as pd
import os
import os.path
from pprint import pprint
from IPython.display import clear_output


In [3]:
PHEN_START = 0
PHEN_END = 10
ICD10_FILE_PATH = '/cs/labs/michall/roeizucker/phewas_run/phecode_icd10.csv'
WORK_PATH = '/cs/labs/michall/roeizucker/phewas_run/runs1/'
PHEN_PER_DIR = 10

In [9]:
# !cat /cs/labs/michall/roeizucker/phewas_run/phecode_icd10.csv

In [6]:
# input  - icd10 list file

file1 = pd.read_csv(ICD10_FILE_PATH)
file1 = file1.sort_values(by=["PheCode"])
# print(file1)
icd10_codes_list = []
current_code_list = []
old_id = -1
old_name = ""
# PheCode
for id in file1.index:
    curr_id = file1["PheCode"][id]
    curr_name = file1["Phenotype"][id]
    if curr_id == 38:
        print(curr_id)
#         input()
    if file1["PheCode"][id] != old_id and old_id != -1 and str(old_id) != "nan":
        icd10_codes_list.append({'name': "Phecode_" + str(old_id), 'source': "ICD-10", 'codings': current_code_list})
        icd10_codes_list.append(
            {'name': "Phecode_" + str(old_id) + "_F", 'source': "ICD-10", 'codings': current_code_list, 'sex_filter': 'F'}, )
        icd10_codes_list.append(
            {'name': "Phecode_" + str(old_id) + "_M", 'source': "ICD-10", 'codings': current_code_list, 'sex_filter': 'M'}, )
        current_code_list = []
    if str(file1["PheCode"][id]) == "nan":
        icd10_codes_list.append(
            {'name': "ICD10_" + file1["ICD10"][id].replace(".", ""), 'source': "ICD-10",
             'codings': [file1["ICD10"][id].replace(".", "")]})
        icd10_codes_list.append(
            {'name': "ICD10_" + file1["ICD10"][id].replace(".", "") + "_F", 'source': "ICD-10",
             'codings': [file1["ICD10"][id].replace(".", "")], 'sex_filter': 'F'})
        icd10_codes_list.append(
            {'name': "ICD10_" + file1["ICD10"][id].replace(".", "") + "_M", 'source': "ICD-10",
             'codings': [file1["ICD10"][id].replace(".", "")], 'sex_filter': 'M'})
    old_id = curr_id
    old_name = curr_name
    current_code_list.append(file1["ICD10"][id].replace(".", ""))



38.0
38.0
38.0
38.0
38.0


In [24]:
curr_phen = []
for i in range(PHEN_START,PHEN_END):
    curr_phen.append(icd10_codes_list[i])
#     print(icd10_codes_list[i])
# print("specs = ",curr_phen)
with open(WORK_PATH + "/file.py", 'w') as fp:
    fp.write("specs = "+ str(curr_phen))


In [19]:
# creates the scripts and folders
counter = 0
while counter < len(icd10_codes_list):
#     create dir for current batch of phenotypes
    dir_name = WORK_PATH + str(counter) + ":" + str(counter + PHEN_PER_DIR - 1)
#     print(dir_name)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
#     print(dir_name)
    curr_phenotypes = []
    for i in range(counter,counter + PHEN_PER_DIR):
        if i >= len(icd10_codes_list):
            break
        phen = icd10_codes_list[i]
        curr_phenotypes.append(phen)
#         create dir for phenotype_raw_results
        phenotype_raw_results = dir_name + "/" + phen["name"]
        if not os.path.exists(phenotype_raw_results):
            os.mkdir(phenotype_raw_results)
            
#         create script_to run phenotype
        command = "source ~/my_python/bin/activate\npwas_test_genes --resolve-quasi-complete-covariate-separation --remove-multicollinear-covariate --dataset-file=" + dir_name + "/ukbb_dataset.csv --gene-effect-scores-dir=/cs/labs/michall/roeizucker/virt_env_install_test/ukbb_imputation_gene_effect_scores/ --per-gene-pwas-results-dir="+phenotype_raw_results+" --sample-id-col=eid --phenotype-col='" + phen["name"] + "' --covariate-cols-json-file=" + dir_name + "/ukbb_covariate_columns.json --task-index-env-variable=SLURM_ARRAY_TASK_ID --total-tasks-env-variable=SLURM_ARRAY_TASK_COUNT\n"
        run_script_path = dir_name + "/run_"+ phen["name"] +".sh"

        if not os.path.exists(run_script_path) and not os.path.exists('/cs/labs/michall/roeizucker/phewas_run/results/'+phen["name"]+'.csv'):
            with open(run_script_path, 'w') as fp:
    #         create run file
                fp.write(command)
    #         give run privlige to script
            os.system("chmod 744 " + run_script_path)
        counter+=1    
#     create phenotype_specs file
    if not os.path.exists(dir_name + "/phenotype_specs.py"):
        with open(dir_name + "/phenotype_specs.py", 'w') as fp:
            fp.write("specs = "+ str(curr_phenotypes))
        
#     create script to run all scripts
# TODO: remove all phenotypes that were already run
    run_all_script_path = dir_name + "/run_all.sh"
    if not os.path.exists(run_all_script_path):
        with open(run_all_script_path, 'w') as fp:
            for phen in curr_phenotypes:
                if not os.path.exists('/cs/labs/michall/roeizucker/phewas_run/results/'+phen["name"]+'.csv'):
                    fp.write('sbatch --array=0-999 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="' + dir_name + "/run_"+ phen["name"]+ '.sh"\n')
                    fp.write('sleep .5\n')
        os.system("chmod 744 " + run_all_script_path)
    
#   create script to combine all phenotype results
# TODO: remove all phenotypes that were already run
    combine_all_script_path = dir_name + "/combine_all.sh"
    if not os.path.exists(combine_all_script_path):
        with open(combine_all_script_path, 'w') as fp:
            for phen in curr_phenotypes:
                if not os.path.exists('/cs/labs/michall/roeizucker/phewas_run/results/'+phen["name"]+'.csv'):
                    fp.write('combine_pwas_results --genes-file=~/my_storage/virt_env_install_test/genes_hg19.csv --per-gene-pwas-results-dir='+dir_name + "/" + phen["name"]+'/ --results-file=/cs/labs/michall/roeizucker/phewas_run/results/'+phen["name"]+'.csv\n')
        os.system("chmod 744 " + combine_all_script_path)
print("Done")

/cs/labs/michall/roeizucker/phewas_run/runs1/0:9
/cs/labs/michall/roeizucker/phewas_run/runs1/10:19
/cs/labs/michall/roeizucker/phewas_run/runs1/20:29
/cs/labs/michall/roeizucker/phewas_run/runs1/30:39
/cs/labs/michall/roeizucker/phewas_run/runs1/40:49
/cs/labs/michall/roeizucker/phewas_run/runs1/50:59
/cs/labs/michall/roeizucker/phewas_run/runs1/60:69
/cs/labs/michall/roeizucker/phewas_run/runs1/70:79
/cs/labs/michall/roeizucker/phewas_run/runs1/80:89
/cs/labs/michall/roeizucker/phewas_run/runs1/90:99
/cs/labs/michall/roeizucker/phewas_run/runs1/100:109
/cs/labs/michall/roeizucker/phewas_run/runs1/110:119
/cs/labs/michall/roeizucker/phewas_run/runs1/120:129
/cs/labs/michall/roeizucker/phewas_run/runs1/130:139
/cs/labs/michall/roeizucker/phewas_run/runs1/140:149
/cs/labs/michall/roeizucker/phewas_run/runs1/150:159
/cs/labs/michall/roeizucker/phewas_run/runs1/160:169
/cs/labs/michall/roeizucker/phewas_run/runs1/170:179
/cs/labs/michall/roeizucker/phewas_run/runs1/180:189
/cs/labs/michal

/cs/labs/michall/roeizucker/phewas_run/runs1/1540:1549
/cs/labs/michall/roeizucker/phewas_run/runs1/1550:1559
/cs/labs/michall/roeizucker/phewas_run/runs1/1560:1569
/cs/labs/michall/roeizucker/phewas_run/runs1/1570:1579
/cs/labs/michall/roeizucker/phewas_run/runs1/1580:1589
/cs/labs/michall/roeizucker/phewas_run/runs1/1590:1599
/cs/labs/michall/roeizucker/phewas_run/runs1/1600:1609
/cs/labs/michall/roeizucker/phewas_run/runs1/1610:1619
/cs/labs/michall/roeizucker/phewas_run/runs1/1620:1629
/cs/labs/michall/roeizucker/phewas_run/runs1/1630:1639
/cs/labs/michall/roeizucker/phewas_run/runs1/1640:1649
/cs/labs/michall/roeizucker/phewas_run/runs1/1650:1659
/cs/labs/michall/roeizucker/phewas_run/runs1/1660:1669
/cs/labs/michall/roeizucker/phewas_run/runs1/1670:1679
/cs/labs/michall/roeizucker/phewas_run/runs1/1680:1689
/cs/labs/michall/roeizucker/phewas_run/runs1/1690:1699
/cs/labs/michall/roeizucker/phewas_run/runs1/1700:1709
/cs/labs/michall/roeizucker/phewas_run/runs1/1710:1719
/cs/labs/m

/cs/labs/michall/roeizucker/phewas_run/runs1/3040:3049
/cs/labs/michall/roeizucker/phewas_run/runs1/3050:3059
/cs/labs/michall/roeizucker/phewas_run/runs1/3060:3069
/cs/labs/michall/roeizucker/phewas_run/runs1/3070:3079
/cs/labs/michall/roeizucker/phewas_run/runs1/3080:3089
/cs/labs/michall/roeizucker/phewas_run/runs1/3090:3099
/cs/labs/michall/roeizucker/phewas_run/runs1/3100:3109
/cs/labs/michall/roeizucker/phewas_run/runs1/3110:3119
/cs/labs/michall/roeizucker/phewas_run/runs1/3120:3129
/cs/labs/michall/roeizucker/phewas_run/runs1/3130:3139
/cs/labs/michall/roeizucker/phewas_run/runs1/3140:3149
/cs/labs/michall/roeizucker/phewas_run/runs1/3150:3159
/cs/labs/michall/roeizucker/phewas_run/runs1/3160:3169
/cs/labs/michall/roeizucker/phewas_run/runs1/3170:3179
/cs/labs/michall/roeizucker/phewas_run/runs1/3180:3189
/cs/labs/michall/roeizucker/phewas_run/runs1/3190:3199
/cs/labs/michall/roeizucker/phewas_run/runs1/3200:3209
/cs/labs/michall/roeizucker/phewas_run/runs1/3210:3219
/cs/labs/m

/cs/labs/michall/roeizucker/phewas_run/runs1/4540:4549
/cs/labs/michall/roeizucker/phewas_run/runs1/4550:4559
/cs/labs/michall/roeizucker/phewas_run/runs1/4560:4569
/cs/labs/michall/roeizucker/phewas_run/runs1/4570:4579
/cs/labs/michall/roeizucker/phewas_run/runs1/4580:4589
/cs/labs/michall/roeizucker/phewas_run/runs1/4590:4599
/cs/labs/michall/roeizucker/phewas_run/runs1/4600:4609
/cs/labs/michall/roeizucker/phewas_run/runs1/4610:4619
/cs/labs/michall/roeizucker/phewas_run/runs1/4620:4629
/cs/labs/michall/roeizucker/phewas_run/runs1/4630:4639
/cs/labs/michall/roeizucker/phewas_run/runs1/4640:4649
/cs/labs/michall/roeizucker/phewas_run/runs1/4650:4659
/cs/labs/michall/roeizucker/phewas_run/runs1/4660:4669
/cs/labs/michall/roeizucker/phewas_run/runs1/4670:4679
/cs/labs/michall/roeizucker/phewas_run/runs1/4680:4689
/cs/labs/michall/roeizucker/phewas_run/runs1/4690:4699
/cs/labs/michall/roeizucker/phewas_run/runs1/4700:4709
/cs/labs/michall/roeizucker/phewas_run/runs1/4710:4719
/cs/labs/m

In [118]:
for i in range(40,110):
    print(f"../{i*10}:{i*10 + 9}/combine_all.sh")


../400:409/combine_all.sh
../410:419/combine_all.sh
../420:429/combine_all.sh
../430:439/combine_all.sh
../440:449/combine_all.sh
../450:459/combine_all.sh
../460:469/combine_all.sh
../470:479/combine_all.sh
../480:489/combine_all.sh
../490:499/combine_all.sh
../500:509/combine_all.sh
../510:519/combine_all.sh
../520:529/combine_all.sh
../530:539/combine_all.sh
../540:549/combine_all.sh
../550:559/combine_all.sh
../560:569/combine_all.sh
../570:579/combine_all.sh
../580:589/combine_all.sh
../590:599/combine_all.sh
../600:609/combine_all.sh
../610:619/combine_all.sh
../620:629/combine_all.sh
../630:639/combine_all.sh
../640:649/combine_all.sh
../650:659/combine_all.sh
../660:669/combine_all.sh
../670:679/combine_all.sh
../680:689/combine_all.sh
../690:699/combine_all.sh
../700:709/combine_all.sh
../710:719/combine_all.sh
../720:729/combine_all.sh
../730:739/combine_all.sh
../740:749/combine_all.sh
../750:759/combine_all.sh
../760:769/combine_all.sh
../770:779/combine_all.sh
../780:789/c

In [ ]:
counter = 0
while counter < len(icd10_codes_list):
#     create dir for current batch of phenotypes
    print(str(counter) + ":" + str(counter + PHEN_PER_DIR - 1))
    dir_name = WORK_PATH + str(counter) + ":" + str(counter + PHEN_PER_DIR - 1)
    file_name = dir_name + "/ukbb_dataset.csv"
    if not os.path.exists(file_name ):
        os.system('create_ukbb_phenotype_dataset --phenotype-specs-file=' +dir_name+'/phenotype_specs.py --output-dataset-file='+file_name+' --output-covariates-columns-file=' + dir_name +'/ukbb_covariate_columns.json')
    counter += PHEN_PER_DIR

0:9
10:19
20:29
30:39
40:49
50:59
60:69
70:79
80:89
90:99
100:109
110:119
120:129
130:139
140:149
150:159
160:169
170:179
180:189
190:199
200:209
210:219
220:229
230:239
240:249
250:259
260:269
270:279
280:289
290:299
300:309
310:319
320:329
330:339
340:349
350:359
360:369
370:379
380:389
390:399
400:409
410:419
420:429
430:439
440:449
450:459
460:469
470:479
480:489
490:499
500:509
510:519
520:529
530:539
540:549
550:559
560:569
570:579
580:589
590:599
600:609
610:619
620:629
630:639
640:649
650:659
660:669
670:679
680:689
690:699
700:709
710:719
720:729
730:739
740:749
750:759
760:769
770:779
780:789
790:799
800:809
810:819
820:829
830:839
840:849
850:859
860:869
870:879
880:889
890:899
900:909
910:919
920:929
930:939
940:949
950:959
960:969
970:979
980:989
990:999
1000:1009
1010:1019
1020:1029
1030:1039
1040:1049
1050:1059
1060:1069
1070:1079
1080:1089
1090:1099
1100:1109
1110:1119
1120:1129
1130:1139
1140:1149
1150:1159
1160:1169
1170:1179
1180:1189
1190:1199
1200:1209
1210:1219
12

In [119]:
file_set = set()
dataset = {}
file_list = []

In [121]:
# smart runs
import os
for i in range(200,210):
    curr_path = f'/cs/labs/michall/roeizucker/phewas_run/runs1/{i*PHEN_PER_DIR}:{i*PHEN_PER_DIR + PHEN_PER_DIR - 1}'
#     print(curr_path)
    first = True
    for x in os.walk(curr_path):
        if first:
            first = False
            continue
        num_of_files = len([name for name in os.listdir(x[0] + '/')])
        with open(curr_path + '/ukbb_dataset.csv') as f:
            first_line = f.readline()
        i = 0
        val_to_check = ''
        for val in first_line.split(','):
            i+=1
            if x[0] == curr_path +'/'+ val:
                val_to_check = val
                break
#         checks that the number of ones
        f = os.popen('cat '+ curr_path +'/ukbb_dataset.csv | awk -F , \'{print $' + str(i) +'}\' | grep 1 | wc -l')
        now = f.read()
        num_of_cases = int(now)
        if num_of_files != 17924 and num_of_cases >= 130:
            if x[0] not in file_set:
                file_set.add(x[0])
                file_list.append(x[0])
                dataset[x[0]] = {}
                dataset[x[0]]['name'] = x[0]
                dataset[x[0]]['num_of_cases'] = num_of_cases
#         print(x[0] , num_of_cases)

#         print(num_of_cases)
#         print(num_of_files)
#         print(x[0])

KeyboardInterrupt: 

In [123]:
print("#!/bin/bash\n\nsmart_wait () {\nNUM=`squeue | grep roeizuck | grep '\[' | wc -l`\nwhile [ $NUM -gt 4 ]\ndo\n        NUM=`squeue | grep roeizuck | grep '\[' | wc -l`\n        sleep 10\ndone\n}")

for file in file_set:
#     print(file)
    todo = f"sbatch --array=0-249 --mem=30g -c1 --time=1-0 --killable --requeue --wrap=\"{file}\""
    todo = todo.replace("Phecode", "run_Phecode")[0:-1] + ".sh\""
    print("cd " +file[0:file.rfind("/")])
    print(todo)
    print("echo \"" + file + "\"")
    print("sleep 60")
    print("smart_wait")

#!/bin/bash

smart_wait () {
NUM=`squeue | grep roeizuck | grep '\[' | wc -l`
while [ $NUM -gt 4 ]
do
        NUM=`squeue | grep roeizuck | grep '\[' | wc -l`
        sleep 10
done
}
cd /cs/labs/michall/roeizucker/phewas_run/runs1/2030:2039
sbatch --array=0-249 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="/cs/labs/michall/roeizucker/phewas_run/runs1/2030:2039/run_Phecode_426.9_M.sh"
echo "/cs/labs/michall/roeizucker/phewas_run/runs1/2030:2039/Phecode_426.9_M"
sleep 60
smart_wait
cd /cs/labs/michall/roeizucker/phewas_run/runs1/2060:2069
sbatch --array=0-249 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="/cs/labs/michall/roeizucker/phewas_run/runs1/2060:2069/run_Phecode_427.6_F.sh"
echo "/cs/labs/michall/roeizucker/phewas_run/runs1/2060:2069/Phecode_427.6_F"
sleep 60
smart_wait
cd /cs/labs/michall/roeizucker/phewas_run/runs1/2020:2029
sbatch --array=0-249 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="/cs/labs/michall/roeizucker/phewas_run/runs1/2020:2029/run_Pheco

In [122]:
print(len(file_set))

64


In [30]:
# data per phenotype: 
# name, PhewasID,num of cases, num of significant genes, path to file
phenotypes = []
# data per gene per phenotype:
# code

In [137]:
def get_relevant_genes_num(fname,rate = None):
    with open(fname, 'r') as read_obj:
        counter = 0
        read_obj.readline()
        for line in read_obj:
            if rate is None and line.split(',')[-2] == 'True':
                counter+=1
            elif rate is not None and float(line.split(',')[-1]) <= rate:
                counter+=1
            else:
                return counter

    return -1
output = []
ICD10_FILE_PATH = '/cs/labs/michall/roeizucker/phewas_run/phecode_icd10.csv'
file1 = pd.read_csv(ICD10_FILE_PATH)
file1 = file1.sort_values(by=["PheCode"])
# print(file1)
icd10_codes_list = []
current_code_list = []
val_per_code = {}
old_id = -1
old_name = ""
# PheCode
for id in file1.index:
    curr_id = file1["PheCode"][id]
    curr_name = file1["Phenotype"][id]
#     if curr_id == 38:
#         print(curr_name)
#         input()
    if file1["PheCode"][id] != old_id and old_id != -1 and str(old_id) != "nan":
        icd10_codes_list.append({'name': "Phecode_" + str(old_id), 'source': "ICD-10", 'codings': current_code_list})
        icd10_codes_list.append(
            {'name': "Phecode_" + str(old_id) + "_F", 'source': "ICD-10", 'codings': current_code_list, 'sex_filter': 'F'}, )
        icd10_codes_list.append(
            {'name': "Phecode_" + str(old_id) + "_M", 'source': "ICD-10", 'codings': current_code_list, 'sex_filter': 'M'}, )
        
        val_per_code["Phecode_" + str(float(old_id))] = (curr_name,current_code_list)
        val_per_code["Phecode_" + str(float(old_id))+ "_F"] = (str(curr_name) + " (F)",current_code_list)
        val_per_code["Phecode_" + str(float(old_id))+ "_M"] = (str(curr_name) + " (M)",current_code_list)
        current_code_list = []
    if str(file1["PheCode"][id]) == "nan":
        icd10_codes_list.append(
            {'name': "ICD10_" + file1["ICD10"][id].replace(".", ""), 'source': "ICD-10",
             'codings': [file1["ICD10"][id].replace(".", "")]})
        icd10_codes_list.append(
            {'name': "ICD10_" + file1["ICD10"][id].replace(".", "") + "_F", 'source': "ICD-10",
             'codings': [file1["ICD10"][id].replace(".", "")], 'sex_filter': 'F'})
        icd10_codes_list.append(
            {'name': "ICD10_" + file1["ICD10"][id].replace(".", "") + "_M", 'source': "ICD-10",
             'codings': [file1["ICD10"][id].replace(".", "")], 'sex_filter': 'M'})
        
        
        val_per_code["ICD10_" + file1["ICD10"][id].replace(".", "")] = (file1["ICD10 String"][id],[file1["ICD10"][id].replace(".", "")])
        val_per_code["ICD10_" + file1["ICD10"][id].replace(".", "") + "_F"] = (file1["ICD10 String"][id] + " (F)",[file1["ICD10"][id].replace(".", "")])
        val_per_code["ICD10_" + file1["ICD10"][id].replace(".", "") + "_M"] = (file1["ICD10 String"][id] + " (M)",[file1["ICD10"][id].replace(".", "")])
    old_id = curr_id
    old_name = curr_name
    current_code_list.append(file1["ICD10"][id].replace(".", ""))


print("name,num_of_cases,num_of_controls,num_relevant_genes,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column")
for i in range(0,510):
    if i == 462:
        continue
    curr_path = f'/cs/labs/michall/roeizucker/phewas_run/runs1/{i*PHEN_PER_DIR}:{i*PHEN_PER_DIR + PHEN_PER_DIR - 1}'
    first = True
#     iterate for each directory, each directory represents a phenotype
    for x in os.walk(curr_path):
        if first:
            first = False
            continue
        if '/cs/labs/michall/roeizucker/phewas_run/runs1/0:9/plink_run' in x[0]:
            continue
        num_of_files = len([name for name in os.listdir(x[0] + '/')])
        with open(curr_path + '/ukbb_dataset.csv') as f:
            first_line = f.readline()
        i = 0
        val_to_check = ''
        for val in first_line.split(','):
            i+=1
            if x[0] == curr_path +'/'+ val:
                val_to_check = val
                break
        if i > PHEN_PER_DIR + 4:
            continue
#         Find number of cases
        f = os.popen('tail -n +2 '+ curr_path +'/ukbb_dataset.csv | awk -F , \'{print $' + str(i) +'}\' | grep 1 | wc -l')
        now = f.read()
        num_of_cases = int(now)
        f.close()
        
#         Find number of controls
        f = os.popen('tail -n +2 '+ curr_path +'/ukbb_dataset.csv | awk -F , \'{print $' + str(i) +'}\' | grep 0 | wc -l')
        now = f.read()
        num_of_controls = int(now)
        f.close()
        
        phenotype_name = val_to_check
        path = x[0]
        phenotype = {}
        phenotype["name"] = phenotype_name
        phenotype["num_of_cases"] = num_of_cases
        phenotype["num_of_controls"] = num_of_controls
        phenotype["num_relevant_genes"] = -1
        phenotype["result_file_path"] = ""
        result_file_path = f'/cs/labs/michall/roeizucker/phewas_run/results/{phenotype["name"]}.csv'
        if os.path.isfile(result_file_path):
            phenotype["num_relevant_genes"] = get_relevant_genes_num(result_file_path)
            phenotype["result_file_path"] = result_file_path
        phenotype["sex"] = "B"
#         find Phenotype Sex
        if phenotype["name"][-1] == 'F':
            phenotype["sex"] = 'F'
        if phenotype["name"][-1] == 'M':
            phenotype["sex"] = 'M'
        key = phenotype['name']
        icd_list = str(val_per_code[key][1]).replace(",",";")
#         output the data
        val = f"{str(phenotype['name']).replace(',',';')},{phenotype['num_of_cases']},{phenotype['num_of_controls']},{phenotype['num_relevant_genes']},{phenotype['result_file_path']},{phenotype['sex']},{str(val_per_code[key][0]).replace(',',';')},{icd_list},Binary,phewas,{curr_path }/ukbb_dataset.csv,{i}"
        print(val, flush=True)
        if val not in output:
            output.append(val)


name,num_of_cases,num_of_controls,num_relevant_genes,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column
Phecode_8.51_F,20,149183,-1,,F,Intestinal infection due to C. difficile (F),['A040'; 'A041'; 'A043'; 'A044'; 'A042'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/0:9/ukbb_dataset.csv,10
Phecode_8.5,1766,274824,11,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_8.5.csv,B,Intestinal e.coli,['A031'; 'A029'; 'A028'; 'A022'; 'A02'; 'A058'; 'A030'; 'A010'; 'A020'; 'A054'; 'A03'; 'A052'; 'A051'; 'A032'; 'A050'; 'A049'; 'A048'; 'A033'; 'A046'; 'A045'; 'A038'; 'A039'; 'A053'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/0:9/ukbb_dataset.csv,6
Phecode_8.0_M,3106,125641,0,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_8.0_M.csv,M,Bacterial enteritis (M),['A00'; 'A060'; 'A062'; 'A063'; 'A065'; 'A066'; 'A067'; 'A06'; 'A068'; 'A07'; 'A073'; 'A079'; 'A09'; 'A222'; 'A213'; 'A069'; 'A059'; 'A064'; 'A01'; 'A011'; 'A012'; '

Phecode_31.1,0,274824,-1,,B,Septicemia,['A304'; 'A301'; 'A305'; 'A308'; 'A309'; 'A302'; 'A303'; 'A30'; 'A300'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/20:29/ukbb_dataset.csv,7
Phecode_38.2_M,319,125641,6,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_38.2_M.csv,M,Bacterial infection NOS (M),['A227'; 'A403'; 'A412'; 'A410'; 'A411'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/30:39/ukbb_dataset.csv,8
Phecode_41.0_F,3098,149183,1,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_41.0_F.csv,F,Staphylococcus infections (F),['A422'; 'A208'; 'A42'; 'A159'; 'A217'; 'A218'; 'A219'; 'A22'; 'A228'; 'A399'; 'A398'; 'A229'; 'A23'; 'A230'; 'A391'; 'A428'; 'A39'; 'A429'; 'J201'; 'A209'; 'A21'; 'A203'; 'B479'; 'B471'; 'B47'; 'A200'; 'J200'; 'A499'; 'A498'; 'A493'; 'A492'; 'A49'; 'A488'; 'A484'; 'A483'; 'A482'; 'A480'; 'A20'; 'A439'; 'A438'; 'A43'; 'A379'; 'A259'; 'A421'; 'A233'; 'A269'; 'A238'; 'B967'; 'B966'; 'B963'; 'B965'; 'B964'; 'A239'; 'A25'; 'A24';

Phecode_54.0,278,274824,46,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_54.0.csv,B,Viral hepatitis,['B002'; 'A600'; 'A601'; 'A609'; 'B082'; 'B00'; 'A60'; 'B001'; 'B009'; 'B008'; 'B000'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/50:59/ukbb_dataset.csv,10
Phecode_54.0_F,147,149183,30,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_54.0_F.csv,F,Viral hepatitis (F),['B002'; 'A600'; 'A601'; 'A609'; 'B082'; 'B00'; 'A60'; 'B001'; 'B009'; 'B008'; 'B000'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/50:59/ukbb_dataset.csv,11
Phecode_53.0_F,258,149183,23,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_53.0_F.csv,F,Herpes zoster with nervous system complications (F),['B02'; 'B027'; 'B028'; 'B029'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/50:59/ukbb_dataset.csv,5
Phecode_53.1_F,40,149183,-1,,F,Herpes simplex (F),['G530'; 'B022'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/50:59/ukbb_dataset.csv,8
Phecode_70.1_F

Phecode_71.1,108,274824,-1,,B,Viral warts & HPV,['B227'; 'B222'; 'B218'; 'B201'; 'B232'; 'B238'; 'B24'; 'B200'; 'B230'; 'B213'; 'B220'; 'B202'; 'B203'; 'B212'; 'B211'; 'B208'; 'B204'; 'B206'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/80:89/ukbb_dataset.csv,4
Phecode_78.0_F,497,149183,0,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_78.0_F.csv,F,Viral infection (F),['A630'; 'B07'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/80:89/ukbb_dataset.csv,8
Phecode_79.0_F,1511,149183,0,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_79.0_F.csv,F,Varicella infection (F),['A932'; 'A938'; 'B263'; 'A94'; 'A950'; 'A931'; 'A951'; 'A959'; 'A96'; 'A960'; 'A961'; 'A962'; 'A95'; 'A930'; 'A921'; 'A929'; 'A920'; 'A928'; 'B342'; 'A924'; 'B343'; 'A923'; 'B341'; 'A968'; 'A90'; 'A91'; 'A92'; 'B269'; 'A93'; 'B340'; 'A988'; 'B260'; 'B05'; 'B051'; 'B054'; 'B058'; 'B059'; 'B06'; 'B060'; 'B068'; 'B04'; 'B069'; 'B081'; 'B083'; 'B084'; 'B085'; 'B088'; 'B338'; 'B09'; 'B268

Phecode_110.1_M,188,125641,27,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_110.1_M.csv,M,Dermatophytosis of nail (M),['B35'; 'B369'; 'B359'; 'B350'; 'B352'; 'B358'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/110:119/ukbb_dataset.csv,6
Phecode_112.3,37,274824,-1,,B,Mycoses,['B372'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/120:129/ukbb_dataset.csv,12
Phecode_110.2_F,21,149183,-1,,F,Candidiasis (F),['B360'; 'B363'; 'B368'; 'B361'; 'B362'; 'B36'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/120:129/ukbb_dataset.csv,7
Phecode_110.13_M,21,125641,-1,,M,Dermatomycoses (M),['B356'; 'B355'; 'B354'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/120:129/ukbb_dataset.csv,5
Phecode_112.0_M,1028,125641,0,/cs/labs/michall/roeizucker/phewas_run/results/Phecode_112.0_M.csv,M,Candidiasis of skin and nails (M),['B379'; 'B378'; 'B377'; 'B374'; 'B373'; 'B204'; 'B370'; 'B37'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/120:

...

ICD10_N049,141,274824,-1,,B,Nephrotic syndrome; unspecified,['N049'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4880:4889/ukbb_dataset.csv,7
ICD10_N04,169,274824,-1,,B,Nephrotic syndrome,['N04'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4880:4889/ukbb_dataset.csv,4
ICD10_N049_M,88,125641,-1,,M,Nephrotic syndrome; unspecified (M),['N049'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4880:4889/ukbb_dataset.csv,9
ICD10_N009_M,12,125641,-1,,M,Acute nephritic syndrome; Unspecified (M),['N009'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4880:4889/ukbb_dataset.csv,3
ICD10_N04_M,102,125641,-1,,M,Nephrotic syndrome (M),['N04'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4880:4889/ukbb_dataset.csv,6
ICD10_N188,53,274824,-1,,B,Other chronic renal failure,['N188'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4880:4889/ukbb_dataset.csv,10
ICD10_N188_F,19,149183,-1,,F,Other chronic renal failure (F),['N188'],

ICD10_T935_F,15,149183,-1,,F,Sequelae of injury of muscle and tendon of lower limb (F),['T935'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4930:4939/ukbb_dataset.csv,6
ICD10_T925_F,25,149183,-1,,F,Sequelae of injury of muscle and tendon of upper limb (F),['T925'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4930:4939/ukbb_dataset.csv,3
ICD10_T935,30,274824,-1,,B,Sequelae of injury of muscle and tendon of lower limb,['T935'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4930:4939/ukbb_dataset.csv,5
ICD10_Y407,13,274824,-1,,B,Adverse effects in the therapeutic use of antifungal antibiotics; systemically used,['Y407'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4930:4939/ukbb_dataset.csv,8
ICD10_Y407_M,8,125641,-1,,M,Adverse effects in the therapeutic use of antifungal antibiotics; systemically used (M),['Y407'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4930:4939/ukbb_dataset.csv,10
ICD10_T925_M,45,125641,-1,,M,Seq

ICD10_Z280,1,274824,-1,,B,Immunization not carried out because of contraindication,['Z280'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4970:4979/ukbb_dataset.csv,10
ICD10_Z280_M,0,125641,-1,,M,Immunization not carried out because of contraindication (M),['Z280'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4970:4979/ukbb_dataset.csv,12
ICD10_Z229_F,2,149183,-1,,F,Carrier of infectious disease; unspecified (F),['Z229'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4970:4979/ukbb_dataset.csv,5
ICD10_Z28_F,4,149183,-1,,F,Immunization not carried out (F),['Z28'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4970:4979/ukbb_dataset.csv,8
ICD10_Z282_F,0,149183,-1,,F,Immunization not carried out because of patient's decision for other and unspecified reasons (F),['Z282'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/4980:4989/ukbb_dataset.csv,4
ICD10_Z288_F,1,149183,-1,,F,Immunization not carried out for other reasons (F),['Z

ICD10_Z64_F,44,149183,-1,,F,Problems related to certain psychosocial circumstances (F),['Z64'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5020:5029/ukbb_dataset.csv,3
ICD10_Z640_M,1,125641,-1,,M,Problems related to unwanted pregnancy (M),['Z640'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5020:5029/ukbb_dataset.csv,7
ICD10_Z641,0,274824,-1,,B,Problems related to multiparity,['Z641'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5020:5029/ukbb_dataset.csv,8
ICD10_Z641_M,0,125641,-1,,M,Problems related to multiparity (M),['Z641'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5020:5029/ukbb_dataset.csv,10
ICD10_Z650_M,0,125641,-1,,M,Conviction in civil and criminal proceedings without imprisonment (M),['Z650'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5030:5039/ukbb_dataset.csv,9
ICD10_Z650,0,274824,-1,,B,Conviction in civil and criminal proceedings without imprisonment,['Z650'],Binary,phewas,/cs/labs/michall/roeiz

ICD10_Z732_M,0,125641,-1,,M,Lack of relaxation and leisure (M),['Z732'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5070:5079/ukbb_dataset.csv,5
ICD10_Z734_F,0,149183,-1,,F,Inadequate social skills; not elsewhere classified (F),['Z734'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5070:5079/ukbb_dataset.csv,10
ICD10_Z733_M,105,125641,-1,,M,Stress; not elsewhere classified (M),['Z733'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5070:5079/ukbb_dataset.csv,8
ICD10_Z732,1,274824,-1,,B,Lack of relaxation and leisure,['Z732'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5070:5079/ukbb_dataset.csv,3
ICD10_Z735,0,274824,-1,,B,Social role conflict; not elsewhere classified,['Z735'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5070:5079/ukbb_dataset.csv,12
ICD10_Z736_F,17,149183,-1,,F,Limitation of activities due to disability (F),['Z736'],Binary,phewas,/cs/labs/michall/roeizucker/phewas_run/runs1/5080:5089/ukbb_dataset.csv

In [2]:
print(len(output))

NameError: name 'output' is not defined

In [129]:
with open('/cs/labs/michall/roeizucker/phewas_run/discription4.csv', 'a') as the_file:
    for line in output:
        the_file.write(line + "\n")